In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# read results

In [2]:
peaks = pd.read_pickle('../PEAKS_Results_KAIKO/all_results_from_peaks.pkl')
novor = pd.read_pickle('../Novor_Results_KAIKO/all_results_from_novor.pkl')
kaiko = pd.read_pickle('../PnnlRun3_235_v4_0001_ep_60/mgf_test/all_results_df.pkl')

In [3]:
kaiko = kaiko[kaiko.file_id>234]

print(peaks.shape)
print(novor.shape)
print(kaiko.shape)

(511765, 11)
(511765, 9)
(511765, 17)


In [6]:
kaiko.head()

,accuracy_AA,accuracy_AA_lbyl,exact_match,file_id,len_AA,match_score,output_score,output_seq,scan,scan_number,target_seq,len_output_AA,longest_match_idx_with_novor,longest_match_length_with_novor,longest_match_idx_inv,longest_match_length_inv,longest_match
0,8,8,0,235,10,16,-0.19,"R,L,G,E,H,T,Mmod,E,E,R",235:471,471,"L,R,G,E,H,T,Mmod,E,E,R",10,2,8,0,8,8
1,8,8,1,235,8,16,-0.07,"L,H,H,E,V,T,A,K",235:536,536,"L,H,H,E,V,T,A,K",8,0,8,0,8,8
2,1,0,0,235,10,1,-0.76,"R,S,E,K,G,A,E,R,P,S,R",235:697,697,"A,K,D,D,R,D,Q,L,R,R",11,9,1,0,1,1
3,4,0,0,235,9,4,-0.35,"S,G,L,G,D,R,L,H,E,R",235:766,766,"R,T,E,Q,A,L,H,E,R",10,5,4,0,4,4
4,10,10,1,235,10,20,-0.28,"N,H,V,A,L,H,P,D,K,K",235:836,836,"N,H,V,A,L,H,P,D,K,K",10,0,10,0,10,10


In [4]:
fig, ax = plt.subplots(figsize=(12,8))
colors = sns.color_palette("Paired", 3)
sns.barplot(x="len_AA", y="exact_match", data=kaiko[kaiko.len_AA<26], ax=ax, linewidth=0, ci=None, color=colors[2], label='Kaiko-4.5M')
sns.barplot(x="len_AA", y="exact_match", data=peaks[peaks.len_AA<26], ax=ax, linewidth=0, ci=None, color=colors[1], label='PEAKS')
sns.barplot(x="len_AA", y="exact_match", data=novor[novor.len_AA<26], ax=ax, linewidth=0, ci=None, color=colors[0], label='Novor')

ax.set(xlabel="AA length")
ax.set(ylabel="Exact match")
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("acc_by_len_peaks.png", dpi=600)

In [ ]:
def plot_longest_substring_length(df, len_AA):
    plt.close("all")
    ax = sns.countplot(x="longest_match_length_with_novor", hue="Method", linewidth=0, data=df[df.len_AA==len_AA],
                       hue_order=['Kaiko-4.5M','PEAKS','Novor'], palette="Paired")
    ax.set(xlabel="Longest match length (len={0})".format(len_AA))
    ax.set(ylabel="Counts")
    plt.legend(frameon=False)
    plt.tight_layout()
    plt.savefig("longest_match_length_{0}.png".format(len_AA), dpi=600)
    #plt.show()

In [ ]:
peaks['Method'] = 'PEAKS'
novor['Method'] = 'Novor'
kaiko['Method'] = 'Kaiko-4.5M'

df = pd.concat([peaks[['longest_match_length_with_novor','len_AA','Method']],
                novor[['longest_match_length_with_novor','len_AA','Method']],
                kaiko[['longest_match_length_with_novor','len_AA','Method']]
               ], ignore_index=True)
df.head()

In [ ]:
for i in [8,9,12,15,17,20]:
    plot_longest_substring_length(df, i)

In [5]:
mass_H = 1.0078
mass_H2O = 18.0106
mass_NH3 = 17.0265
mass_N_terminus = 1.0078
mass_C_terminus = 17.0027
mass_CO = 27.9949

mass_AA = {'_PAD': 0.0,
           '_GO': mass_N_terminus-mass_H,
           '_EOS': mass_C_terminus+mass_H,
           'A': 71.03711, # 0
           'R': 156.10111, # 1
           'N': 114.04293, # 2
           'Nmod': 115.02695,
           'D': 115.02694, # 3
           'C': 103.00919, # 4
           'Cmod': 160.03065, # C(+57.02)
           #~ 'Cmod': 161.01919, # C(+58.01) # orbi
           'E': 129.04259, # 5
           'Q': 128.05858, # 6
           'Qmod': 129.0426,
           'G': 57.02146, # 7
           'H': 137.05891, # 8
           'I': 113.08406, # 9
           'L': 113.08406, # 10
           'K': 128.09496, # 11
           'M': 131.04049, # 12
           'Mmod': 147.0354,
           'F': 147.06841, # 13
           'P': 97.05276, # 14
           'S': 87.03203, # 15
           'T': 101.04768, # 16
           'W': 186.07931, # 17
           'Y': 163.06333, # 18
           'V': 99.06841, # 19
          }

def get_longest_match(output_seq, target_seq, len_AA, inv=False):
#     if exact_match > 0:
#         return (0, len_AA)
    if (type(output_seq) == float) | (output_seq == ""):
        return (0, 0)
    if inv:
        output = output_seq.split(',')[::-1]
        decoder_input = target_seq.split(',')[::-1]
    else:
        output = output_seq.split(',')
        decoder_input = target_seq.split(',')
    
    decoder_input_len = len(decoder_input)
    output_len = len(output)
    decoder_input_mass = [mass_AA[x] for x in decoder_input]
    decoder_input_mass_cum = np.cumsum(decoder_input_mass)
    output_mass = [mass_AA[x] for x in output]
    output_mass_cum = np.cumsum(output_mass)
    
    length = min(decoder_input_len, output_len)
    
    num_match = 0
    i = 0
    j = 0
    match_list = [0 for i in range(decoder_input_len)]
    
    while i < decoder_input_len and j < output_len:
        if abs(decoder_input_mass_cum[i] - output_mass_cum[j]) < 0.5:
            if abs(decoder_input_mass[i] - output_mass[j]) < 0.1:
                num_match += 1
                match_list[i] = 1
            i += 1
            j += 1
        elif decoder_input_mass_cum[i] < output_mass_cum[j]:
            i += 1
        else:
            j += 1
    #print('out put_seq:', output)
    #print('target_seq:', decoder_input)
    #print(match_list)
    count_list = match_list
    for i in range(length-1):
        if count_list[length - i - 2] == 0:
            count_list[length - i - 2] = 0
        else:
            count_list[length - i - 2] += count_list[length - i - 1]
    #print(count_list)
    max_idx = np.argmax(count_list)
    return (max_idx, count_list[max_idx])

get_longest_match("V,M,Mmod,E,A,S,H,H,K,P,G,K", "V,Mmod,E,A,S,H,H,K,P,G,K", 11, False)
get_longest_match("V,M,Mmod,E,A,S,H,H,K,P,G,K", "V,Mmod,E,A,S,H,H,K,P,G,K", 11, True)

(0, 10)

In [7]:
peaks[['longest_match_idx_inv', 'longest_match_length_inv']] = peaks.apply(
    lambda row: pd.Series(get_longest_match(row['output_seq'], row['target_seq'], row['len_AA'], inv=True), dtype='int32'), axis=1)
novor[['longest_match_idx_inv', 'longest_match_length_inv']] = novor.apply(
    lambda row: pd.Series(get_longest_match(row['output_seq'], row['target_seq'], row['len_AA'], inv=True), dtype='int32'), axis=1)
# kaiko[['longest_match_idx_inv', 'longest_match_length_inv']] = kaiko.apply(
#     lambda row: pd.Series(get_longest_match(row['output_seq'], row['target_seq'], row['len_AA'], inv=True), dtype='int32'), axis=1)

In [8]:
print(np.sum(peaks.longest_match_length_inv>peaks.longest_match_length_with_novor))
print(np.sum(novor.longest_match_length_inv>novor.longest_match_length_with_novor))
print(np.sum(kaiko.longest_match_length_inv>kaiko.longest_match_length_with_novor))

139083
203535
120957


In [9]:
novor['longest_match'] = novor[['longest_match_length_inv', 'longest_match_length_with_novor']].max(axis=1)
peaks['longest_match'] = peaks[['longest_match_length_inv', 'longest_match_length_with_novor']].max(axis=1)
# kaiko['longest_match'] = kaiko[['longest_match_length_inv', 'longest_match_length_with_novor']].max(axis=1)

In [ ]:
sns.palplot(sns.color_palette("Paired", n_colors=3, desat=.5))
plt.show()
sns.palplot(sns.color_palette("Paired", n_colors=3))
plt.show()

In [ ]:
def plot_longest_match(df, len_AA):
    plt.close("all")
    tf = df[df.len_AA==len_AA]
    
    ax = sns.countplot(x="longest_match", hue="Method", linewidth=0, data=tf,
                       hue_order=['Kaiko-4.5M','PEAKS','Novor'], palette="Paired")
#     ax = sns.countplot(x="longest_match", hue="Method", linewidth=0, data=tf[tf.longest_match_length_with_novor>=tf.longest_match_length_inv],
#                        hue_order=['Kaiko-4.5M','PEAKS','Novor'], palette=sns.color_palette("Paired", n_colors=3, desat=.5, ))
    ax.set(xlabel="Longest match length (len={0})".format(len_AA))
    ax.set(ylabel="Counts")
    
    tf.groupby(by="Method").longest_match
    
    ax2 = ax.twinx()
    
    plt.legend(frameon=False)
    plt.tight_layout()
#     plt.savefig("longest_match_{0}.png".format(len_AA), dpi=600)
    plt.show()


for i in [8,9,12,15,17,20]:
    plot_longest_match(df, i)

In [ ]:
peaks['Method'] = 'PEAKS'
novor['Method'] = 'Novor'
kaiko['Method'] = 'Kaiko-4.5M'

In [ ]:
def plot_longest_match_with_cumsum(df, len_AA):
    tot, _ = df[(df.len_AA==len_AA)&(df.Method=="PEAKS")].shape
    ttf = df[df.len_AA==len_AA].groupby(by=["Method",'longest_match']).len_AA.agg('count').unstack(level=-1)
    ttf = ttf[ttf.columns[::-1]]
    ttfc = ttf.cumsum(axis=1)
    print(ttf)
    ttfc.columns = ttfc.columns[::-1]
    print(ttfc)
    ttfc = 100.0*ttfc/tot

    fig, ax = plt.subplots(figsize=(10,6))

    ttf.transpose()[['Kaiko-4.5M','PEAKS','Novor']].plot(kind='bar', ax=ax, legend=True)#, colormap=plt.get_cmap("Paired"))
    ttfc.transpose()[['Kaiko-4.5M','PEAKS','Novor']].plot(kind='line', secondary_y=True, ax=ax, legend=True)#, colormap=plt.get_cmap("Paired"))
    ax.legend(loc='center right')
    plt.savefig("longest_match_with_cumsum_{0}.png".format(len_AA), dpi=600)
#     plt.show()
# plot_longest_match_with_cumsum(df, 8)

for i in [8,9,12,15,17,20]:
    plot_longest_match_with_cumsum(df, i)

In [ ]:
kaiko[(kaiko.longest_match_length_inv>kaiko.longest_match_length_with_novor)&(kaiko.longest_match_length_with_novor==0)][['output_seq','target_seq','longest_match_length_with_novor','longest_match_length_inv']]

In [ ]:
ss = 0
for i in 'V,A,V,N,R'.split(','):
    ss += mass_AA[i]
ss

In [23]:
peaks.head()

,Scan,Peptide,Length,id,scan,target_seq,len_AA,output_seq,longest_match_idx_with_novor,longest_match_length_with_novor,exact_match,longest_match_idx_inv,longest_match_length_inv,longest_match,file_id,scan_number
0,215.0,VM(+15.99)FNYHPKGK,10.0,237.0,237:215,"V,Mmod,E,A,S,H,H,K,P,G,K",11,"V,Mmod,F,N,Y,H,P,K,G,K",0,2,0,0,2,2,237,215
1,228.0,YTGPDHHKPGK,11.0,237.0,237:228,"V,Mmod,E,A,S,H,H,K,P,G,K",11,"Y,T,G,P,D,H,H,K,P,G,K",5,6,0,0,6,6,237,228
2,309.0,VMTWHHKGPK,10.0,237.0,237:309,"V,M,E,A,S,H,H,K,P,G,K",11,"V,M,T,W,H,H,K,G,P,K",5,3,0,3,3,3,237,309
3,401.0,VMEASHHKPGK,11.0,237.0,237:401,"V,M,E,A,S,H,H,K,P,G,K",11,"V,M,E,A,S,H,H,K,P,G,K",0,11,1,0,11,11,237,401
4,408.0,HYHMVHLDGK,10.0,237.0,237:408,"V,Mmod,E,A,S,H,H,K,P,G,K",11,"H,Y,H,M,V,H,L,D,G,K",9,1,0,0,2,2,237,408


In [22]:
peaks[['file_id', "scan_number"]] = pd.DataFrame(peaks.scan.str.split(':').tolist(), columns=['file_id', "scan_number"], dtype="int32")

In [24]:
del peaks['id']

In [ ]:
kaiko[(kaiko.longest_match_length_inv==0)&(kaiko.longest_match_length_with_novor==0)][['output_seq','target_seq','longest_match_length_with_novor','longest_match_length_inv']]

In [25]:
novor.to_pickle('../Novor_Results_KAIKO/all_results_from_novor.pkl')
peaks.to_pickle('../PEAKS_Results_KAIKO/all_results_from_peaks.pkl')

# common spectra in three

In [10]:
kaiko_correct_scans = kaiko[kaiko.exact_match==1].scan.tolist()
peaks_correct_scans = peaks[peaks.exact_match==1].scan.tolist()
novor_correct_scans = novor[novor.exact_match==1].scan.tolist()

print('Correct Scans in Kaiko:', kaiko[kaiko.exact_match==1].shape[0])
print('Correct Scans in PEAKS:', peaks[peaks.exact_match==1].shape[0])
print('Correct Scans in Novor:', novor[novor.exact_match==1].shape[0])

print('Correct Scans in PEAKS and Kaiko:', peaks[(peaks.scan.isin(kaiko_correct_scans))&(peaks.exact_match==1)].shape[0])
print('Correct Scans in Novor and Kaiko:', novor[(novor.scan.isin(kaiko_correct_scans))&(novor.exact_match==1)].shape[0])
print('Correct Scans in PEAKS and Novor:', peaks[(peaks.scan.isin(novor_correct_scans))&(peaks.exact_match==1)].shape[0])

print('Correct Scans in one of three:', np.unique(kaiko_correct_scans+peaks_correct_scans+novor_correct_scans).shape)
print('Correct Scans in all three:', kaiko[(kaiko.scan.isin(peaks_correct_scans))&(kaiko.scan.isin(novor_correct_scans))&(kaiko.exact_match==1)].shape[0])

Correct Scans in Kaiko: 166761
Correct Scans in PEAKS: 136477
Correct Scans in Novor: 100784
Correct Scans in PEAKS and Kaiko: 94403
Correct Scans in Novor and Kaiko: 84866
Correct Scans in PEAKS and Novor: 69733
Correct Scans in one of three: (217463,)
Correct Scans in all three: 62443


In [11]:
def find_common(_kaiko, _peaks, _novor, len_aa):
    tkaiko = _kaiko[_kaiko.len_AA==len_aa]
    tpeaks = _peaks[_peaks.len_AA==len_aa]
    tnovor = _novor[_novor.len_AA==len_aa]
    
    kaiko_correct_scans = tkaiko[tkaiko.exact_match==1].scan.tolist()
    peaks_correct_scans = tpeaks[tpeaks.exact_match==1].scan.tolist()
    novor_correct_scans = tnovor[tnovor.exact_match==1].scan.tolist()

    print('Correct Scans in Kaiko:', tkaiko[tkaiko.exact_match==1].shape[0])
    print('Correct Scans in PEAKS:', tpeaks[tpeaks.exact_match==1].shape[0])
    print('Correct Scans in Novor:', tnovor[tnovor.exact_match==1].shape[0])

    print('Correct Scans in PEAKS and Kaiko:', tpeaks[(tpeaks.scan.isin(kaiko_correct_scans))&(tpeaks.exact_match==1)].shape[0])
    print('Correct Scans in Novor and Kaiko:', tnovor[(tnovor.scan.isin(kaiko_correct_scans))&(tnovor.exact_match==1)].shape[0])
    print('Correct Scans in PEAKS and Novor:', tpeaks[(tpeaks.scan.isin(novor_correct_scans))&(tpeaks.exact_match==1)].shape[0])

    print('Correct Scans in one of three:', np.unique(kaiko_correct_scans+peaks_correct_scans+novor_correct_scans).shape)
    print('Correct Scans in all three:', tkaiko[(tkaiko.scan.isin(peaks_correct_scans))&(tkaiko.scan.isin(novor_correct_scans))&(tkaiko.exact_match==1)].shape[0])

In [12]:
find_common(kaiko, peaks, novor, len_aa=15)

Correct Scans in Kaiko: 10578
Correct Scans in PEAKS: 8318
Correct Scans in Novor: 4694
Correct Scans in PEAKS and Kaiko: 4547
Correct Scans in Novor and Kaiko: 3433
Correct Scans in PEAKS and Novor: 2618
Correct Scans in one of three: (15058,)
Correct Scans in all three: 2066


In [13]:
## hard spectra to identify
min_leng = 2

kaiko_hard_scans = kaiko[kaiko.longest_match<=min_leng].scan.tolist()
peaks_hard_scans = peaks[peaks.longest_match<=min_leng].scan.tolist()
novor_hard_scans = novor[novor.longest_match<=min_leng].scan.tolist()

print('Hard Scans in Kaiko:', len(kaiko_hard_scans))
print('Hard Scans in PEAKS:', len(peaks_hard_scans))
print('Hard Scans in Novor:', len(novor_hard_scans))

print('Hard Scans in both PEAKS and Kaiko:', peaks[(peaks.scan.isin(kaiko_hard_scans))&(peaks.longest_match<=min_leng)].shape[0])
print('Hard Scans in both Novor and Kaiko:', novor[(novor.scan.isin(kaiko_hard_scans))&(novor.longest_match<=min_leng)].shape[0])
print('Hard Scans in both PEAKS and Novor:', peaks[(peaks.scan.isin(novor_hard_scans))&(peaks.longest_match<=min_leng)].shape[0])

print('Hard Scans in one of three:', np.unique(kaiko_hard_scans+peaks_hard_scans+novor_hard_scans).shape)
print('Hard Scans in all three:', kaiko[(kaiko.scan.isin(peaks_hard_scans))&(kaiko.scan.isin(novor_hard_scans))&(kaiko.longest_match<=min_leng)].shape[0])

Hard Scans in Kaiko: 58507
Hard Scans in PEAKS: 50412
Hard Scans in Novor: 54578
Hard Scans in both PEAKS and Kaiko: 24504
Hard Scans in both Novor and Kaiko: 24174
Hard Scans in both PEAKS and Novor: 30745
Hard Scans in one of three: (101570,)
Hard Scans in all three: 17496


In [ ]:
## hard spectra to identify
min_leng = 2

kaiko_hard_scans = kaiko[kaiko.longest_match<=min_leng].scan.tolist()
peaks_hard_scans = peaks[peaks.longest_match<=min_leng].scan.tolist()
novor_hard_scans = novor[novor.longest_match<=min_leng].scan.tolist()

print('Hard Scans in Kaiko:', len(kaiko_hard_scans))
print('Hard Scans in PEAKS:', len(peaks_hard_scans))
print('Hard Scans in Novor:', len(novor_hard_scans))

print('Hard Scans in both PEAKS and Kaiko:', peaks[(peaks.scan.isin(kaiko_hard_scans))&(peaks.longest_match<=min_leng)].shape[0])
print('Hard Scans in both Novor and Kaiko:', novor[(novor.scan.isin(kaiko_hard_scans))&(novor.longest_match<=min_leng)].shape[0])
print('Hard Scans in both PEAKS and Novor:', peaks[(peaks.scan.isin(novor_hard_scans))&(peaks.longest_match<=min_leng)].shape[0])

print('Hard Scans in one of three:', np.unique(kaiko_hard_scans+peaks_hard_scans+novor_hard_scans).shape)
print('Hard Scans in all three:', kaiko[(kaiko.scan.isin(peaks_hard_scans))&(kaiko.scan.isin(novor_hard_scans))&(kaiko.longest_match<=min_leng)].shape[0])